**Animal Performance on W-Track** 

In [24]:
data_directory = '/home/zoldello/Documents/data/L5/r1/20230531_150338/'
trodes_directory = '/home/zoldello/Apps/Trodes_2-4-0_Ubuntu2004/'

file_name = f'{data_directory}20230531_150338.rec'
trodesexport_runner = f'{trodes_directory}trodesexport'

In [25]:
!$trodesexport_runner -rec {file_name} -dio -time
#!/home/zoldello/Apps/Trodes_2-4-0_Ubuntu2004/trodesexport -rec {file_name} -dio -time

Exporting data...
Done7%


In [17]:
#trodes_directory
trodesexport_runner

'/home/zoldello/Apps/Trodes_2-4-0_Ubuntu2004/trodesexport'